In [2]:
import os
import random
import xml.etree.ElementTree as ET
import shutil
import cv2
import matplotlib.pyplot as plt
from utils import *
import albumentations as A
from albumentations.core.transforms_interface import DualTransform
import random
from YOLODetector import YOLODetector
from CharClassifier import CharClassifier
from sklearn.model_selection import train_test_split
import pandas as pd

plt.rcParams["figure.figsize"] = (5,5)

In [18]:
import sys
del sys.modules['utils']

# Generate data

Tạo thư mục data

In [ ]:
os.mkdir("character_recognition/data")

In [3]:
ALPHA_DICT = {0: '0', 1: '1', 2: '2', 3: '3', 4: '4', 5: '5', 6: '6', 7: '7', 8: '8', 9: '9',
    10: 'A', 11: 'B', 12: 'C', 13: 'D', 14: 'E', 15: 'F', 16: 'G', 17: 'H', 18: 'K', 19: 'L',
    20: 'M', 21: 'N', 22: 'P', 23: 'R', 24: 'S', 25: 'T', 26: 'U', 27: 'V', 28: 'X', 29: 'Y', 30: 'Z'}

In [5]:
for c in ALPHA_DICT.values():
    os.mkdir("character_recognition/data/"+c)

Tách chiết data

In [5]:
source_path = "lp_detection/bike_data/"
dest_path = "character_recognition/crop/"

In [6]:
lst_im = os.listdir(source_path)
lst_im = [file for file in lst_im if len(file) > 10 and file[-3:] == 'jpg']
len(lst_im)

1740

In [21]:
for file in lst_im:
    img = cv2.imread(source_path+file)
    box, _ = read_yolo_content(source_path+file[:-3]+"txt")
    box = box[0]
    cv2.imwrite(dest_path+file, crop_im(img, box))
    #break

Sinh ký tự

In [3]:
chr_det = YOLODetector('character_detection/yolov4-tiny-char-detect_last.weights', 'character_detection/yolov4-tiny-char-detect.cfg', confi_thres=0.7)
chr_cls = CharClassifier('character_recognition/myCNN_backup_28_BN.h5')

2021-10-02 22:10:26.334533: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-10-02 22:10:26.334806: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
random.shuffle(lst_im)

In [13]:
char_count = dict(zip(ALPHA_DICT.values(), [0]*len(ALPHA_DICT)))

In [25]:
source_path = "character_recognition/crop/"
dest_path = "character_recognition/data/"
MAX_COUNT = 50

for file in lst_im[1500:]:
    img = cv2.imread(source_path+file)

    crop_resize = resize_with_ratio(img, 100/img.shape[1])
    ori_lp = cv2.GaussianBlur(crop_resize, (5,5), 1.0)
    
    crop_resize = cv2.resize(img, (100, 100))
    resz_lp = cv2.GaussianBlur(crop_resize, (5,5), 1.0)

    bboxes = chr_det.detect(resz_lp)
    bboxes = [list(bbox) for bbox in bboxes]

    characters = []
    for bbox in bboxes:
        pad_bbox = bbox.copy()
        pad_bbox[2] += 0.05
        pad_bbox[3] += 0.05
        new_im = crop_im(ori_lp, pad_bbox)
        new_im = cv2.cvtColor(new_im, cv2.COLOR_BGR2GRAY)
        new_im = resize_with_ratio(new_im, 28/new_im.shape[0])
        border = 28-new_im.shape[1]
        if border % 2 == 0:
            border_L = border_R = border // 2
        else:
            border_L = border // 2
            border_R = border_L + 1
        if border > 0:
            new_im = cv2.copyMakeBorder(new_im, 0, 0, border_L, border_R, cv2.BORDER_REPLICATE)
        else:
            new_im = cv2.resize(new_im, (28,28))
        characters.append(new_im)
    
    pred_chars = chr_cls.predict(characters)

    for c, lab in zip(characters, pred_chars):
        if char_count[lab] == MAX_COUNT:
            continue
        cv2.imwrite(dest_path+lab+"/"+str(char_count[lab])+".jpg", c)
        char_count[lab] += 1

    #break

In [31]:
file_count = 0
for root, subFolders, files in os.walk("character_recognition/data/"):
    for file in files:
        if file[-3:] == 'jpg':
            newname = "char" + str(file_count) + ".jpg"
            os.rename(root+"/"+file, root+"/"+newname)
            file_count += 1

In [33]:
file_count

1275

Chia tập train val

In [4]:
data_ls = []

for root, subFolders, files in os.walk("character_recognition/data/"):
    for file in files:
        if file[-3:] == 'jpg':
            data_ls.append((file, root[-1]))

In [44]:
data_ls_df = pd.DataFrame(data_ls, columns=['Image', "Class"])
data_ls_df

,Image,Class
0,char10.jpg,R
1,char11.jpg,R
2,char13.jpg,R
3,char12.jpg,R
4,char16.jpg,R
...,...,...
1269,char1250.jpg,P
1270,char1246.jpg,P
1271,char1252.jpg,P
1272,char1253.jpg,P


In [45]:
X_train, X_val, y_train, y_val = train_test_split(
    data_ls_df.Image, data_ls_df.Class,
    test_size=0.3,
    random_state=42,
    stratify=data_ls_df.Class
)

In [46]:
train_df = pd.concat([X_train, y_train], axis=1)
val_df = pd.concat([X_val, y_val], axis=1)
train_df.to_csv("character_recognition/train.csv", index=False)
val_df.to_csv("character_recognition/val.csv", index=False)

In [1]:
!mkdir character_recognition/data_splitted

In [4]:
os.mkdir("character_recognition/data_splitted/train/")
os.mkdir("character_recognition/data_splitted/val/")
for c in ALPHA_DICT.values():
    os.mkdir("character_recognition/data_splitted/train/"+c)
    os.mkdir("character_recognition/data_splitted/val/"+c)


In [7]:
train_df = pd.read_csv("character_recognition/train.csv")
val_df = pd.read_csv("character_recognition/val.csv")

In [13]:
dest_path = "character_recognition/data_splitted/"
source_path = "character_recognition/data/"

for item in train_df.values:
    name, lab = list(item)
    shutil.copyfile(source_path+lab+"/"+name, dest_path+"train/"+lab+"/"+name)
    #break

for item in val_df.values:
    name, lab = list(item)
    shutil.copyfile(source_path+lab+"/"+name, dest_path+"val/"+lab+"/"+name)